# 📊📄📈 PyWin32 Office 자동화 실습 노트북 (C:\ 루트 저장 + PowerPoint)

이 노트북은 Windows 환경에서 **C 드라이브 루트**에 파일을 저장하며, PyWin32를 사용해
- Excel 자동화 3가지
- PowerPoint 성적표 자동 생성
- Word 자동화 3가지
- Outlook Gemini API 자동 답장

예제를 포함합니다.


In [ ]:
# 필요한 패키지 설치 (최초 1회)
# !pip install pywin32 google-genai pandas openpyxl
print("패키지 설치 확인")

In [ ]:
# 📈 Excel 예제 1: 새 통합 문서 생성 → C:\학생점수.xlsx 저장
import win32com.client as win32

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Add()
ws = wb.Worksheets('Sheet1')

data = [['이름','국어','영어','수학'],['철수',90,85,92],['영희',88,79,95],['민수',75,80,78]]
for r, row in enumerate(data, start=1):
    for c, val in enumerate(row, start=1):
        ws.Cells(r,c).Value = val

path = r'C:\\학생점수.xlsx'
wb.SaveAs(path)
print("저장:", path)
wb.Close(False); excel.Quit()

In [ ]:
# 📈 Excel 예제 2: C:\학생점수.xlsx 열기 → B2 +5 점 → 저장
import win32com.client as win32, os

path = r'C:\\학생점수.xlsx'
if not os.path.exists(path):
    raise FileNotFoundError(path + " 없음")

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(path)
ws = wb.Worksheets('Sheet1')

orig = ws.Range("B2").Value
ws.Range("B2").Value = orig + 5
wb.Save()
print("B2 업데이트:", ws.Range("B2").Value)

wb.Close(False); excel.Quit()

In [ ]:
# 📈 Excel 예제 3: C:\학생점수.xlsx 차트 생성 → C:\chart.png
import win32com.client as win32, os

path = r'C:\\학생점수.xlsx'
if not os.path.exists(path):
    raise FileNotFoundError(path + " 없음")

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(path); ws = wb.Worksheets('Sheet1')

chart = wb.Charts.Add()
chart.ChartType = 51  # 막대형
chart.SetSourceData(Source=ws.Range("B2:D4"))
chart.HasTitle = True
chart.ChartTitle.Text = "학생별 점수"
img = r'C:\\chart.png'
chart.Export(img)
print("차트 저장:", img)

wb.Close(False); excel.Quit()

## 📊 PowerPoint 자동화 예제 – 성적표 슬라이드 생성

Excel 예제에서 만든 **C:\학생점수.xlsx**를 읽어
PowerPoint 성적표 슬라이드를 자동 생성합니다.

In [ ]:
# PowerPoint 성적표 자동 생성
import win32com.client as win32
import pandas as pd

excel_path = r'C:\\학생점수.xlsx'
ppt_path   = r'C:\\GradeReport.pptx'

df = pd.read_excel(excel_path)
ppt = win32.gencache.EnsureDispatch('PowerPoint.Application')
ppt.Visible = True
pres = ppt.Presentations.Add()
slide = pres.Slides.Add(1, 1)  # 제목 슬라이드
slide.Shapes.Title.TextFrame.TextRange.Text = "학생 성적표"

table = slide.Shapes.AddTable(df.shape[0]+1, df.shape[1], 50,150,600,300).Table
# 헤더
for col, name in enumerate(df.columns, start=1):
    table.Cell(1,col).Shape.TextFrame.TextRange.Text = name
# 데이터
for r in range(df.shape[0]):
    for c, val in enumerate(df.iloc[r], start=1):
        table.Cell(r+2,c).Shape.TextFrame.TextRange.Text = str(val)

pres.SaveAs(ppt_path)
print("PowerPoint 저장:", ppt_path)

In [ ]:
# 📄 Word 예제 1: C:\AutoReport.docx 생성
import win32com.client as win32
word = win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc = word.Documents.Add()
sel = word.Selection
sel.Style = doc.Styles("제목 1"); sel.TypeText("자동 보고서\n\n")
sel.Style = doc.Styles("본문"); sel.TypeText("PyWin32 자동 생성 문서")
path = r'C:\\AutoReport.docx'; doc.SaveAs(path)
print("저장:",path)
doc.Close(False); word.Quit()

In [ ]:
# 📄 Word 예제 2: 차트 삽입
import win32com.client as win32, os
doc_path=r'C:\\AutoReport.docx'; img=r'C:\\chart.png'
word=win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc=word.Documents.Open(doc_path); sel=word.Selection; sel.EndKey(6)
sel.InlineShapes.AddPicture(img); sel.ParagraphFormat.Alignment=1
doc.Save(); print("이미지 삽입 완료")
doc.Close(False); word.Quit()

In [ ]:
# 📄 Word 예제 3: '보고서'→'리포트', C:\AutoReport.pdf 저장
import win32com.client as win32
word=win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc=word.Documents.Open(r'C:\\AutoReport.docx')
find=word.Selection.Find; find.Text="보고서"; find.Replacement.Text="리포트"; find.Execute(Replace=2)
path=r'C:\\AutoReport.pdf'; doc.SaveAs(path, FileFormat=17)
print("PDF 저장:",path)
doc.Close(False); word.Quit()

In [ ]:
# ✉️ Outlook 자동 답장 (Gemini API)
import win32com.client as win32
from google import genai; from google.genai import types
client=genai.Client(api_key='GEMINI_API_KEY')
model="gemini-2.5-pro-preview-03-25"
outlook=win32.gencache.EnsureDispatch("Outlook.Application")
sel=outlook.ActiveExplorer().Selection
if sel.Count==0: raise RuntimeError("메일 선택 필요")
mail=sel.Item(1); orig=mail.Body
prompt=f"공손하게 답장 (한국어):\n\n{orig}"
contents=[types.Content(role="user",parts=[types.Part.from_text(prompt)])]
cfg=types.GenerateContentConfig(response_mime_type="text/plain")
reply_text=""
for ch in client.models.generate_content_stream(model=model,contents=contents,config=cfg):
    reply_text+=ch.text
rep=mail.Reply(); rep.Body=reply_text+"\n\n(자동생성)"
rep.Display()
print("답장 생성")